In [1]:
import torch
import torch.nn as nn
import numpy as np
import cvxpy as cp

In [2]:
torch.__version__

'1.11.0'

In [3]:
np.__version__

'1.20.3'

In [4]:
torch.manual_seed(0)
np.random.seed(0)

In [5]:
D = 2000
N = 500

In [6]:
X = np.random.randn(D, N)
w = np.random.rand(N)
z = np.random.rand(N)>0.1
w = w*z
w = w/w.sum()
y = X.dot(w).reshape(-1,1)

In [7]:
X = torch.tensor(X.astype(np.float32))
y = torch.tensor(y.astype(np.float32))

In [8]:
# meta dataset: used for training the lambda producer
X_meta = X[:1900].cuda()
y_meta = y[:1900].cuda()
# training set: used for training the regression model
X_trn = X[1150:1900].cuda()
y_trn = y[1150:1900].cuda()
# test test
X_tst = X[1900:].cuda()
y_tst = y[1900:].cuda()

In [9]:
class Lambda_Producer(nn.Module):
    def __init__(self, D):
        super(Lambda_Producer, self).__init__()
        self.self_attention = nn.MultiheadAttention(D, 1)
        self.fc1 = nn.Linear(D*3, D)
        self.fc2 = nn.Linear(D, 1)

    def forward(self, x, y):
        N = x.shape[1]
        x = x.t()[:,None,:]
        v = self.self_attention(x, x, x, need_weights=False)
        v = torch.cat([x.squeeze(1), v[0].squeeze(1), y.repeat(1, N).t()], axis=1)
        v = self.fc1(v)
        v = torch.relu(v)
        v = self.fc2(v)
        v = torch.exp(v)
        return v

In [10]:
def qp_solver_cvxpy(lambdas, X, y):
    
    N = X.shape[1]
    y = y.flatten()

    P = 2*X.T.dot(X)
    q = -2*X.T.dot(y) + lambdas
    G = -np.eye(N)
    h = np.zeros(N)
    A = np.ones((1,N))
    b = np.array([1.0])

    w = cp.Variable(N)
    
    obj = cp.Minimize((1/2)*cp.quad_form(w, P) + q @ w)
    
    constraints = [A @ w == b, G @ w <= h]
    prob = cp.Problem(obj, constraints)
    prob.solve()
    
    return w.value, constraints[0].dual_value, constraints[1].dual_value, P, q


def diff_w_lambdas(w, a2, P, grad_output):
    D = w.shape[0]
    dfds = torch.vstack([torch.hstack([P, 
                                       torch.ones((D,1)).type_as(P), 
                                       -torch.eye(D).type_as(P)]),
                         torch.hstack([torch.ones((1,D)).type_as(P), 
                                       torch.zeros((1,1)).type_as(P), 
                                       torch.zeros((1,D)).type_as(P)]),
                         torch.hstack([-torch.diag(a2), 
                                       torch.zeros((D,1)).type_as(P), 
                                       -torch.diag(w)])])
    partial = -torch.linalg.solve(dfds.t(), 
                                  torch.vstack([grad_output.view(-1,1), 
                                                torch.zeros((D+1,1)).type_as(P)]))
    return partial[:D].view(-1)


class QPSolver(torch.autograd.Function):

    @staticmethod
    def forward(ctx, lambdas, X, y):
        X_np = X.detach().cpu().numpy()
        y_np = y.detach().cpu().numpy()
        lambdas_np = lambdas.detach().cpu().numpy()
        w, _, a2, P, _ = qp_solver_cvxpy(lambdas_np, X_np, y_np)
        w = torch.from_numpy(w).type_as(X)
        a2 = torch.from_numpy(a2).type_as(X)
        P = torch.from_numpy(P).type_as(X)
        ctx.save_for_backward(w, a2, P)
        return w

    @staticmethod
    def backward(ctx, grad_output):
        w, a2, P = ctx.saved_tensors
        partial = diff_w_lambdas(w, a2, P, grad_output)
        return partial, None, None
    
qp_solver = QPSolver.apply

In [11]:
# 750: meta_trn/trn length
lambda_producer = Lambda_Producer(750).cuda()

In [12]:
opt = torch.optim.Adam(lambda_producer.parameters(), lr=1e-4)

In [13]:
# now we train the lambda producer (i.e., meta model)
# note that, the training set contains 750 days data (that's the period that we use for training the regression model)
# and the testing set contains 100 days data (that's the period that we buy at beginning and hold) 
# thus, we should always sample 750 + 100 days data from meta dataset, and build the meta_train set and meta_test set.

for I in range(200):
    rnd_starting_idx = np.random.choice(1050)
    X_meta_trn = X_meta[rnd_starting_idx:(rnd_starting_idx+750)]
    y_meta_trn = y_meta[rnd_starting_idx:(rnd_starting_idx+750)]
    X_meta_tst = X_meta[(rnd_starting_idx+750):(rnd_starting_idx+850)]
    y_meta_tst = y_meta[(rnd_starting_idx+750):(rnd_starting_idx+850)]
    lambdas = lambda_producer(X_meta_trn, y_meta_trn).flatten()
    w_hat = qp_solver(lambdas, X_meta_trn, y_meta_trn)
    meta_loss = ((X_meta_tst.mm(w_hat.view(-1,1))-y_meta_tst)**2).mean()
    opt.zero_grad()
    meta_loss.backward()
    opt.step()
    if I % 10 == 0:
        print(meta_loss.detach().cpu().numpy())

1.6017117e-05
1.1148499e-05
1.3522909e-05
1.2993502e-05
1.1634161e-05
4.7232315e-06
8.846097e-06
7.21329e-06
7.385661e-06
2.0199393e-06
5.9200775e-06
6.414053e-06
4.8644447e-06
7.188181e-06
5.5105384e-06
4.0770637e-06
3.7563307e-06
3.694881e-06
3.5604696e-06
2.4388557e-06


In [14]:
lambdas = lambda_producer(X_trn, y_trn).flatten().detach()

In [15]:
w_hat = qp_solver(lambdas, X_trn, y_trn)

In [16]:
mse_test = ((X_tst.mm(w_hat.view(-1,1))-y_tst)**2).mean()

In [17]:
mse_test

tensor(2.3518e-06, device='cuda:0')